In [1]:
import pandas as pd
import os
from utils import *
from config import exclude_codes
import datetime
import statsmodels.api as sm
from scipy.optimize import minimize, LinearConstraint
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
from statsmodels.stats.correlation_tools import cov_nearest
from calc_rets_and_cov import get_factor_rets, get_cov_date
from optimize import *


In [32]:
import importlib
import utils
importlib.reload(utils)
from utils import *

In [ ]:
load_from_data = True

In [5]:
weights = fill_weights()
history = get_history_data()
market_cap = get_market_cap()
industry_stock = read_data('industry_stock.csv')
history = pd.merge(history.reset_index(), 
                   market_cap.reset_index(), 
                   how='left', 
                   left_on=['time', 'code'], 
                   right_on=['time', 'code']).set_index('time')
history['ret'] = history.groupby('code')['close'].pct_change(fill_method=None)
history.tail()

,code,close,factor,volume,market_cap,ret
time,,,,,,
2023-07-31,688981.XSHG,51.36,1.0,25944137.0,4070.7610,0.021886
2023-08-01,688981.XSHG,50.72,1.0,18110314.0,4020.0349,-0.012461
2023-08-02,688981.XSHG,50.83,1.0,10813980.0,4028.7534,0.002169
2023-08-03,688981.XSHG,50.91,1.0,12048176.0,4035.0942,0.001574
2023-08-04,688981.XSHG,51.10,1.0,17100810.0,4050.2263,0.003732


In [6]:
factor_folder = './data/factor'
factor_files = os.listdir(factor_folder)
factor_df_map = {}
for file in factor_files:
    df_temp = pd.read_csv(os.path.join('./data/factor', file), index_col=0)
    df_temp.fillna(method='ffill', limit=10, inplace=True)
    df_temp.index = pd.to_datetime(df_temp.index)
    factor_df_map[file[:-4]] = df_temp
    
norm_df_map = {}
for key, value in factor_df_map.items():
    norm_df_map[key] = get_normalized_factor(value, weights)    
    
norm_factor_exposure = pd.concat([value.stack(dropna=False).sort_index(level=1).rename(key) for key, value in norm_df_map.items()], axis=1)
norm_factor_exposure = norm_factor_exposure.reset_index(level=1).rename(columns={'level_1': 'code'})
norm_factor_exposure = norm_factor_exposure[norm_factor_exposure.index >= weights.index[0]]

In [14]:
factor_columns = ['country'] + list(norm_df_map.keys()) + industry_stock.stack().unique().tolist()
factor_rets = pd.DataFrame(index=history.index.unique(), columns=factor_columns)
rsquare_df = pd.Series(index=history.index.unique(), name='rsquare', dtype=float)
residual_rets = pd.DataFrame(index=history.index.unique(), columns=weights.columns)

if load_from_data:
    factor_rets = pd.read_csv('inter_data/factor_rets.csv', index_col=0, parse_dates=True)
    residual_rets = pd.read_csv('inter_data/residual_rets.csv', index_col=0, parse_dates=True)
else:
    dates = history.index.unique()
    dates = dates[dates >= weights.index.min()]

    dates_to_chek = []

    for i in range(0, len(dates)):
        date = dates[i]
        factor_temp, residual_temp, rsquare = get_factor_rets(date, history, weights, norm_df_map, industry_stock)
        factor_rets.loc[date, factor_temp.columns] = factor_temp.values
        residual_rets.loc[date, residual_temp.columns] = residual_temp.values
        rsquare_df.loc[date] = rsquare
        if np.isnan(rsquare):
            print(f'Warning: {date} has nan rsquare')
            dates_to_chek.append(date)
        
        if i % 300 == 0:        
            print(f'Finished the {date}, the rsquare is {rsquare:.2f}')
            
    factor_rets.to_csv('inter_data/factor_rets.csv')
    residual_rets.to_csv('inter_data/residual_rets.csv')        

In [19]:
factor_cov = factor_rets.ewm(halflife=22, min_periods=5).cov(pairwise=True)
residual_vol = residual_rets.ewm(halflife=22, min_periods=5).std()

In [21]:
from statsmodels.regression.rolling import RollingOLS, RollingWLS

# get expected returns
if load_from_data:
    expected_rets_all = pd.read_csv('inter_data/expected_rets.csv', index_col=0, parse_dates=True)
else:
    short_time = 5
    long_time = 30
    ret_df = history.groupby('code')['ret']
    temp = ret_df.rolling(short_time, min_periods=3).sum() - ret_df.rolling(long_time, min_periods=long_time//2).sum()

    signal = temp.unstack(level=0)
    norm_signal = get_normalized_factor(signal, weights)
    norm_signal_lag = norm_signal.shift(2)
    history_ret = history.groupby('code')['ret']

    ret_fut = history[['code', 'ret']].pivot(columns='code', values='ret').shift(-2)

    expected_rets_all = pd.DataFrame(index=ret_fut.index, columns=ret_fut.columns)
    corr_all = []

    print(np.nanmean(norm_signal.corrwith(ret_fut)))
    for i in range(len(ret_fut.columns)):
        colname = ret_fut.columns[i]
        params = RollingWLS(ret_fut.iloc[:,i], 
                            sm.add_constant(norm_signal.iloc[:,i]), 
                            window=30, 
                            min_nobs=20, 
                            ).fit().params
        expected_rets = -(sm.add_constant(norm_signal.iloc[:,i].fillna(0)) * params.shift(2)).sum(axis=1, skipna=False)
        expected_rets_all.loc[:, colname] = expected_rets
        corr_temp = expected_rets.corr(ret_fut.iloc[:,i])
        print(colname, corr_temp)
        corr_all.append(corr_temp)
        
    expected_rets_all.to_csv('inter_data/expected_rets.csv')

In [139]:
np.nanmean(expected_rets_all.corrwith(ret_fut))

0.010124602283565189

In [145]:
all_trading_dates = expected_rets_all.dropna(how='all').index
all_trading_dates = all_trading_dates[all_trading_dates >= '2006-04-27']
weights_optimize_df = pd.DataFrame(0.0, index=all_trading_dates, columns=weights.columns)

In [147]:
cvxopt.solvers.options['show_progress'] = False
model_type = 'cvxopt'
for date in weights_optimize_df.index[weights_optimize_df.sum(axis=1) == 0]:
    factor_cov_date, expected_rets, weight_target, nonzero_codes, tradable_codes, \
    added_stock, remove_stock, non_tradable_codes, date_index, previous_weight, index_weight = \
        prepare_data_for_optimization(date, 
                                  history,
                                  weights,
                                  weights_optimize_df,
                                  factor_cov,
                                  norm_factor_exposure,
                                  residual_vol,
                                  industry_stock,
                                  expected_rets_all)
    if len(remove_stock):
        weight_target += weights_optimize_df.iloc[date_index-1][remove_stock].sum()
        
    if model_type == 'cvxopt':
        if len(remove_stock) > 20:            
            result = optimize_portfolio_cvxopt(expected_rets, factor_cov_date, previous_weight.loc[tradable_codes], (index_weight/index_weight.sum()).loc[tradable_codes], 
                                        target_weight=weight_target, max_turnover=0.15 + weights_optimize_df.iloc[date_index-1][remove_stock].sum() / 2)
        else:
            result = optimize_portfolio_cvxopt(expected_rets, factor_cov_date, previous_weight.loc[tradable_codes], (index_weight/index_weight.sum()).loc[tradable_codes], 
                                    target_weight=weight_target)
        realized_sum = np.array(result['x'][:(len(result['x']) // 3)]).sum() + previous_weight.loc[non_tradable_codes].sum()
        status = result['status'] 
        minimize_value = result['primal objective']
                
    else:    
        if len(remove_stock) > 20:
            result = optimize_portfolio(expected_rets.fillna(0), factor_cov_date.fillna(0), previous_weight.loc[tradable_codes], (index_weight/index_weight.sum()).loc[tradable_codes], 
                                        target_weight=weight_target, max_turnover=0.15 + weights_optimize_df.iloc[date_index-1][remove_stock].sum() / 2)
        else:
            result = optimize_portfolio(expected_rets.fillna(0), factor_cov_date.fillna(0), previous_weight.loc[tradable_codes], (index_weight/index_weight.sum()).loc[tradable_codes], 
                                        target_weight=weight_target)
        realized_sum = result['x'][:(len(result['x']) // 3)].sum() + previous_weight.loc[non_tradable_codes].sum()
        status = result['success']
        minimize_value = result['fun']
        
    print(f'{date}, {status}, {minimize_value:.3f},  Finished, Sum of weight is: {realized_sum:.3f}')
            
    if status in [False] or np.abs(realized_sum - 1) > 1e-3:
        break
        
    weights_optimize_df.loc[date, tradable_codes] = np.array(result['x'][:(len(result['x']) // 3)]).flatten()
    weights_optimize_df.loc[date, non_tradable_codes] = previous_weight.loc[non_tradable_codes]
    


2006-04-27 00:00:00, optimal, -0.001,  Finished, Sum of weight is: 1.000
2006-04-28 00:00:00, optimal, -0.002,  Finished, Sum of weight is: 1.000
2006-05-08 00:00:00, optimal, -0.001,  Finished, Sum of weight is: 1.000
2006-05-09 00:00:00, optimal, -0.000,  Finished, Sum of weight is: 1.000
2006-05-10 00:00:00, optimal, -0.001,  Finished, Sum of weight is: 1.000
2006-05-11 00:00:00, optimal, -0.001,  Finished, Sum of weight is: 1.000
2006-05-12 00:00:00, optimal, -0.000,  Finished, Sum of weight is: 1.000
2006-05-15 00:00:00, optimal, -0.001,  Finished, Sum of weight is: 1.000
2006-05-16 00:00:00, optimal, -0.003,  Finished, Sum of weight is: 1.000
2006-05-17 00:00:00, optimal, -0.003,  Finished, Sum of weight is: 1.000
2006-05-18 00:00:00, optimal, -0.004,  Finished, Sum of weight is: 1.000
2006-05-19 00:00:00, optimal, -0.004,  Finished, Sum of weight is: 1.000
2006-05-22 00:00:00, optimal, -0.011,  Finished, Sum of weight is: 1.000
2006-05-23 00:00:00, optimal, -0.187,  Finished, Su